In [ ]:
import functions.helper

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import Lib and Def Func

In [ ]:
import pandas as pd
import time
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support, \
    confusion_matrix
import torch
import time
import numpy as np
from sklearn.model_selection import train_test_split ,KFold
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier ,GradientBoostingClassifier ,AdaBoostClassifier ,ExtraTreesClassifier ,VotingClassifier ,StackingClassifier
 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
 
import xgboost as xgb

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline 

In [ ]:
## load all data set in to a dataframe dictionary (UPDATE :NORMALIZED)

all_df=get_all_df_dict(normalize=False ,normalize_feature_list=[])

In [ ]:
# Train using all the features 
print("Summary\n Class 0- True news \n Class 1 - False news\n")
print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp',"dataset","model"))  # correct


classifiers=[
          #  ['Logistic Regression',LogisticRegression()],
    #  ['Decision Tree Classification',DecisionTreeClassifier()],
    #  ['Gradient Boosting Classification', GradientBoostingClassifier()],
    #  ['Ada Boosting Classification',AdaBoostClassifier()],
    #  ['Extra Tree Classification', ExtraTreesClassifier(n_estimators=300)],
    #  ['K-Neighbors Classification',KNeighborsClassifier()],
    #  ['Support Vector Classification',SVC()],
      ['Gaussian Naive Bayes',GaussianNB()],
    #  ["xgBoost",xgb.XGBClassifier()]
]

for key,value in all_df.items(): #for each dataset in all_df
  # print("----------",key,"--------------")
  df = value
  if (key=="codalab" or key=="liar"):
    df_train = df.loc[df["split_Sementic"]=="train"][All_features+['label']]
    df_test = df.loc[df["split_Sementic"]!="train"][All_features+['label']]  

    X_train=df_train[All_features]
    y_train=df_train["label"]
    X_val=df_test[All_features]
    y_val=df_test["label"]
  else:
    X_train, X_val, y_train, y_val = train_test_split(df[All_features], df["label"], test_size=0.3, random_state=142,stratify=df["label"])
  

  compare_models( X_train, X_val, y_train, y_val ,All_features,key ,classifiers,) 

Summary
 Class 0- True news 
 Class 1 - False news

prec-t	prec-f	rec-t	rec-f	f1-t	f1-f	accu	tn	fp	fn	tp	dataset	model
0.633	0.781	0.892	0.428	0.740	0.553	0.672	1990	242	1154	864	codalab	Gaussian Naive Bayes
0.770	0.458	0.955	0.117	0.853	0.186	0.751	4705	220	1404	186	fakenewsnet	Gaussian Naive Bayes
0.999	0.998	0.998	0.999	0.999	0.999	0.999	6414	11	5	6812	isot	Gaussian Naive Bayes
0.972	0.987	0.987	0.971	0.980	0.979	0.979	940	12	27	910	kagglerealfake	Gaussian Naive Bayes
0.564	0.569	0.875	0.196	0.686	0.291	0.565	1206	172	933	227	liar	Gaussian Naive Bayes


In [ ]:
# load models 
models={}
baseModelPath= outputDirectory
predictionScore= []

# trained_models_path={
#   "codalab":baseModelPath+"codalab"+"_Extra Tree Classification.pkl",
#   "fakenewsnet": baseModelPath+"fakenewsnet"+"_Extra Tree Classification.pkl",
#   "isot": baseModelPath+"isot"+"_Extra Tree Classification.pkl",
#   "kagglerealfake": baseModelPath+"kagglerealfake"+"_Extra Tree Classification.pkl",
#   "liar":baseModelPath+"liar"+"_Extra Tree Classification.pkl",
# }

trained_models_path={
  "codalab":baseModelPath+"codalab"+"_Gaussian Naive Bayes.pkl",
  "fakenewsnet": baseModelPath+"fakenewsnet"+"_Gaussian Naive Bayes.pkl",
  "isot": baseModelPath+"isot"+"_Gaussian Naive Bayes.pkl",
  "kagglerealfake": baseModelPath+"kagglerealfake"+"_Gaussian Naive Bayes.pkl",
  "liar":baseModelPath+"liar"+"_Gaussian Naive Bayes.pkl",
}
 

print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp',"dataset","model"))  # correct


# need to load the model one at a time to avoid crash 
for key1,value1 in all_df.items():
  df = value1
  X_val=df[All_features]
  y_val=df["label"]
  for key2,value in trained_models_path.items(): 
    Path =value
    value2=read_pickle_model(Path)
    predict_for_model(X_val,y_val,All_features,key1,value2,key2)

prec-t	prec-f	rec-t	rec-f	f1-t	f1-f	accu	tn	fp	fn	tp	dataset	model
0.637	0.801	0.904	0.428	0.747	0.558	0.678	5048	535	2880	2157	codalab			codalab
0.360	0.342	0.304	0.400	0.330	0.369	0.350	1700	3883	3021	2016	codalab			fakenewsnet
0.256	0.472	0.004	0.987	0.008	0.639	0.470	22	5561	64	4973	codalab			isot
0.278	0.442	0.061	0.826	0.099	0.576	0.424	338	5245	876	4161	codalab			kagglerealfake
0.598	0.566	0.637	0.526	0.617	0.546	0.584	3554	2029	2386	2651	codalab			liar
0.792	0.304	0.652	0.470	0.715	0.369	0.607	10694	5720	2807	2494	fakenewsnet			codalab
0.772	0.504	0.961	0.124	0.856	0.198	0.756	15769	645	4646	655	fakenewsnet			fakenewsnet
0.851	0.244	0.003	0.998	0.007	0.393	0.246	57	16357	10	5291	fakenewsnet			isot
0.788	0.246	0.071	0.941	0.130	0.391	0.283	1159	15255	312	4989	fakenewsnet			kagglerealfake
0.812	0.291	0.487	0.651	0.609	0.402	0.527	7995	8419	1850	3451	fakenewsnet			liar
0.575	0.870	0.946	0.340	0.715	0.489	0.634	20265	1151	15002	7722	isot			codalab
0.799	0.936	0.943	0.776	0.865	0.84

# Voting Classifier

In [ ]:
# Train using all the features | Voting claasifier 
n_estimators=300
voting="soft"
print("Summary\n Class 0- True news \n Class 1 - False news\n")
print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp',"dataset","model"))  # correct

for key,value in all_df.items():
  # print("----------",key,"--------------")
  df = value
  if (key=="codalab" or key=="liar"):
    df_train = df.loc[df["split_Sementic"]=="train"][All_features+['label']]
    df_test = df.loc[df["split_Sementic"]!="train"][All_features+['label']]  

    X_train=df_train[All_features]
    y_train=df_train["label"]
    X_val=df_test[All_features]
    y_val=df_test["label"]
  else:
    X_train, X_val, y_train, y_val = train_test_split(df[All_features], df["label"], test_size=0.3, random_state=142,stratify=df["label"])
  

  # column transforemers 
  sementic_ct = ColumnTransformer([("sementic","passthrough", Sementic_features )])
  lex_ct_embed_ct = ColumnTransformer([("lexicon","passthrough", LexMod_Features +Embed_features)])
  Emotion_ct = ColumnTransformer([("emotion","passthrough", Emotion_features )])
  # embed_ct = ColumnTransformer([("embedding","passthrough", embed_features )])

  
  GaussianNB
  # def model for each estimators
  model_sementic_clf = GaussianNB() #  ExtraTreesClassifier(n_estimators=n_estimators)
  model_lex_emd_clf =  GaussianNB() # ExtraTreesClassifier(n_estimators=n_estimators)
  model_emotion_clf =  GaussianNB() #ExtraTreesClassifier(n_estimators=n_estimators)
  # model_embd_pipeline = ExtraTreesClassifier()
  # final_estimator=ExtraTreesClassifier()


# create pipeline 
  sementic_pipeline = Pipeline([
        ('trans', sementic_ct),
        ('clf', model_sementic_clf)
        ]
  )
  lex_embd_pipeline = Pipeline([
    ('trans', lex_ct_embed_ct),
    ('clf', model_lex_emd_clf )
    ]
  )
  emotion_pipeline = Pipeline([
    ('trans', Emotion_ct),
    ('clf', model_emotion_clf)
    ]
  )
  
  
  estimators = [
    ('sementic_estimator', sementic_pipeline),
    ('lexP_embd_estimator', lex_embd_pipeline),
    ("emotion_estimator" ,emotion_pipeline )
  ]

  final_classifier =VotingClassifier(estimators=estimators ,voting=voting) 
  final_classifier.fit(X_train, y_train)
  write_to_pickle(outputDirectory+key+"_"+"VotingClassifier_"+voting+".pkl",final_classifier) 
  predicted_y= final_classifier.predict(X_val)
  d=compute_metrics(predicted_y,y_val)

  tn, fp, fn, tp = d["confusiton_mat"].ravel() #correct
  print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp,key,"ExtraTree"+voting))

Summary
 Class 0- True news 
 Class 1 - False news

prec-t	prec-f	rec-t	rec-f	f1-t	f1-f	accu	tn	fp	fn	tp	dataset	model
0.612	0.802	0.921	0.354	0.735	0.491	0.652	2055	177	1303	715	codalab	ExtraTreesoft
0.763	0.514	0.983	0.056	0.859	0.101	0.757	4841	84	1501	89	fakenewsnet	ExtraTreesoft
0.828	0.985	0.987	0.806	0.900	0.887	0.894	6340	85	1321	5496	isot	ExtraTreesoft
0.721	0.979	0.987	0.612	0.833	0.753	0.801	940	12	364	573	kagglerealfake	ExtraTreesoft
0.557	0.588	0.929	0.121	0.696	0.200	0.559	1280	98	1020	140	liar	ExtraTreesoft


In [ ]:
models={}
baseModelPath= outputDirectory

trained_models_path={
  "codalab":baseModelPath+"codalab",
  "fakenewsnet": baseModelPath+"fakenewsnet",
  "isot": baseModelPath+"isot",
  "kagglerealfake": baseModelPath+"kagglerealfake",
  "liar":baseModelPath+"liar",
}
 
print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp',"dataset","model"))  # correct

for key1,value1 in all_df.items():
  df = value1
  X_val=df[All_features]
  y_val=df["label"]
  for key2,value in trained_models_path.items():
    Path =value+"_VotingClassifier_"+voting+ ".pkl" 
    value2=read_pickle_model(Path)
    predicted_y= value2.predict(X_val)
    d=compute_metrics(predicted_y,y_val)

    tn, fp, fn, tp = d["confusiton_mat"].ravel() #correct
    print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp,key1,key2))


prec-t	prec-f	rec-t	rec-f	f1-t	f1-f	accu	tn	fp	fn	tp	dataset	model
0.616	0.832	0.936	0.353	0.743	0.496	0.659	5224	359	3258	1779	codalab	codalab
0.510	0.437	0.683	0.273	0.584	0.336	0.489	3815	1768	3664	1373	codalab	fakenewsnet
0.589	0.484	0.146	0.887	0.234	0.626	0.497	815	4768	569	4468	codalab	isot
0.548	0.483	0.293	0.732	0.382	0.582	0.502	1637	3946	1348	3689	codalab	kagglerealfake
0.589	0.583	0.709	0.452	0.643	0.509	0.587	3956	1627	2760	2277	codalab	liar
0.787	0.305	0.688	0.425	0.734	0.355	0.623	11285	5129	3047	2254	fakenewsnet	codalab
0.764	0.561	0.985	0.059	0.861	0.106	0.759	16171	243	4990	311	fakenewsnet	fakenewsnet
0.749	0.244	0.050	0.948	0.094	0.388	0.269	820	15594	275	5026	fakenewsnet	isot
0.770	0.246	0.114	0.895	0.198	0.386	0.305	1869	14545	557	4744	fakenewsnet	kagglerealfake
0.803	0.304	0.595	0.550	0.683	0.392	0.584	9760	6654	2388	2913	fakenewsnet	liar
0.505	0.744	0.957	0.117	0.661	0.201	0.525	20505	911	20076	2648	isot	codalab
0.680	0.966	0.979	0.567	0.803	0.714	0.767	20958	458

# Voting Classifier updated

In [ ]:
# Train using all the features | Voting claasifier 
n_estimators=300
voting="hard"
print("Summary\n Class 0- True news \n Class 1 - False news\n")
print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp',"dataset","model"))  # correct

for key,value in all_df.items():
  # print("----------",key,"--------------")
  df = value
  if (key=="codalab" or key=="liar"):
    df_train = df.loc[df["split_Sementic"]=="train"][All_features+['label']]
    df_test = df.loc[df["split_Sementic"]!="train"][All_features+['label']]  

    X_train=df_train[All_features]
    y_train=df_train["label"]
    X_val=df_test[All_features]
    y_val=df_test["label"]
  else:
    X_train, X_val, y_train, y_val = train_test_split(df[All_features], df["label"], test_size=0.3, random_state=142,stratify=df["label"])
  


  estimators=[
      # ('Decision Tree Classification',DecisionTreeClassifier()),
      # ('Gradient Boosting Classification', GradientBoostingClassifier()),
      ('Ada Boosting Classification',AdaBoostClassifier()),
      ('Extra Tree Classification', ExtraTreesClassifier(n_estimators=300)),
      ('Support Vector Classification',SVC(probability=True)),
      ('Gaussian Naive Bayes',GaussianNB()),
      ("xgBoost",xgb.XGBClassifier()),
      # ("extra tree",ExtraTreesClassifier()),
      # ("RandomForestClassifier",RandomForestClassifier())
  ]
  
  # estimators = [
  #   ('sementic_estimator', sementic_pipeline),
  #   ('lexP_embd_estimator', lex_embd_pipeline),
  #   ("emotion_estimator" ,emotion_pipeline )
  # ]

  final_classifier =VotingClassifier(estimators=estimators ,voting=voting) 
  final_classifier.fit(X_train, y_train)
  write_to_pickle(outputDirectory+key+"_"+"VotingClassifier_"+voting+".pkl",final_classifier) 
  predicted_y= final_classifier.predict(X_val)
  d=compute_metrics(predicted_y,y_val)

  tn, fp, fn, tp = d["confusiton_mat"].ravel() #correct
  print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp,key,"ExtraTree"+voting))

Summary
 Class 0- True news 
 Class 1 - False news

prec-t	prec-f	rec-t	rec-f	f1-t	f1-f	accu	tn	fp	fn	tp	dataset	model
0.816	0.851	0.876	0.781	0.845	0.814	0.831	1955	277	442	1576	codalab	ExtraTreehard
0.771	0.693	0.986	0.095	0.866	0.167	0.769	4858	67	1439	151	fakenewsnet	ExtraTreehard
0.999	1.000	1.000	0.999	0.999	0.999	0.999	6422	3	4	6813	isot	ExtraTreehard
0.988	0.981	0.981	0.988	0.985	0.985	0.985	934	18	11	926	kagglerealfake	ExtraTreehard
0.576	0.572	0.823	0.281	0.678	0.377	0.575	1134	244	834	326	liar	ExtraTreehard


In [ ]:
models={}
baseModelPath= outputDirectory

trained_models_path={
  "codalab":baseModelPath+"codalab",
  "fakenewsnet": baseModelPath+"fakenewsnet",
  "isot": baseModelPath+"isot",
  "kagglerealfake": baseModelPath+"kagglerealfake",
  "liar":baseModelPath+"liar",
}
 
print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-t','prec-f', 'rec-t','rec-f','f1-t','f1-f','accu','tn', 'fp', 'fn', 'tp',"dataset","model"))  # correct

for key1,value1 in all_df.items():
  df = value1
  X_val=df[All_features]
  y_val=df["label"]
  for key2,value in trained_models_path.items():
    Path =value+"_VotingClassifier_"+voting+ ".pkl" 
    value2=read_pickle_model(Path)
    predicted_y= value2.predict(X_val)
    d=compute_metrics(predicted_y,y_val)

    tn, fp, fn, tp = d["confusiton_mat"].ravel() #correct
    print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp,key1,key2))


prec-t	prec-f	rec-t	rec-f	f1-t	f1-f	accu	tn	fp	fn	tp	dataset	model
0.847	0.884	0.903	0.820	0.874	0.851	0.864	5043	540	909	4128	codalab	codalab
0.539	0.841	0.989	0.063	0.698	0.117	0.550	5523	60	4719	318	codalab	fakenewsnet
0.492	0.462	0.252	0.711	0.333	0.560	0.470	1408	4175	1454	3583	codalab	isot
0.507	0.463	0.359	0.612	0.420	0.527	0.479	2005	3578	1953	3084	codalab	kagglerealfake
0.582	0.675	0.864	0.313	0.696	0.428	0.603	4822	761	3458	1579	codalab	liar
0.802	0.267	0.356	0.728	0.493	0.391	0.447	5841	10573	1444	3857	fakenewsnet	codalab
0.776	0.786	0.990	0.116	0.870	0.203	0.777	16246	168	4684	617	fakenewsnet	fakenewsnet
0.760	0.246	0.260	0.746	0.387	0.370	0.379	4264	12150	1344	3957	fakenewsnet	isot
0.757	0.245	0.361	0.642	0.489	0.354	0.430	5926	10488	1900	3401	fakenewsnet	kagglerealfake
0.781	0.319	0.771	0.332	0.776	0.325	0.664	12654	3760	3543	1758	fakenewsnet	liar
0.716	0.620	0.463	0.827	0.563	0.709	0.650	9921	11495	3937	18787	isot	codalab
0.564	0.896	0.963	0.298	0.711	0.447	0.621	20631	7